In [1]:
import sys
import os

In [2]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [4]:
import numpy as np
import os
import io
import cv2

import torch

from config import (Config,
                    eval_dict_leaf)

from demo_utils import (retrieve_text,
                  _frame_from_video,
                  setup_internvideo2)

[2024-06-06 17:05:53,301] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [5]:
video = cv2.VideoCapture('example1.mp4')
frames = [x for x in _frame_from_video(video)]

In [6]:
text_candidates = ["A playful dog and its owner wrestle in the snowy yard, chasing each other with joyous abandon.",
                   "A man in a gray coat walks through the snowy landscape, pulling a sleigh loaded with toys.",
                   "A person dressed in a blue jacket shovels the snow-covered pavement outside their house.",
                   "A pet dog excitedly runs through the snowy yard, chasing a toy thrown by its owner.",
                   "A person stands on the snowy floor, pushing a sled loaded with blankets, preparing for a fun-filled ride.",
                   "A man in a gray hat and coat walks through the snowy yard, carefully navigating around the trees.",
                   "A playful dog slides down a snowy hill, wagging its tail with delight.",
                   "A person in a blue jacket walks their pet on a leash, enjoying a peaceful winter walk among the trees.",
                   "A man in a gray sweater plays fetch with his dog in the snowy yard, throwing a toy and watching it run.",
                   "A person bundled up in a blanket walks through the snowy landscape, enjoying the serene winter scenery."]

In [7]:
config = Config.from_file('internvideo2_stage2_config.py')
config = eval_dict_leaf(config)

In [8]:
model_pth = '/work/piyush/pretrained_checkpoints/LargeModels/InternVideo/1B_clip.pth'
config['pretrained_path'] = model_pth

In [16]:
config["model"]["vision_encoder"]["pretrained"] = model_pth

In [21]:
config['TextEncoders']['bert_large']['config'] = "../configs/config_bert_large.json"

In [22]:
intern_model, tokenizer = setup_internvideo2(config)

/users/piyush/install/miniconda3/envs/vl/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


AssertionError: odict_keys(['temp', 'vision_encoder.clip_projector.norm1_q.weight', 'vision_encoder.clip_projector.norm1_q.bias', 'vision_encoder.clip_projector.norm1_k.weight', 'vision_encoder.clip_projector.norm1_k.bias', 'vision_encoder.clip_projector.norm1_v.weight', 'vision_encoder.clip_projector.norm1_v.bias', 'vision_encoder.clip_projector.cross_attn.q_bias', 'vision_encoder.clip_projector.cross_attn.k_bias', 'vision_encoder.clip_projector.cross_attn.v_bias', 'vision_encoder.clip_projector.cross_attn.q.weight', 'vision_encoder.clip_projector.cross_attn.k.weight', 'vision_encoder.clip_projector.cross_attn.v.weight', 'vision_encoder.clip_projector.cross_attn.proj.weight', 'vision_encoder.clip_projector.cross_attn.proj.bias', 'text_encoder.transformer.layers.0.self_attn.rotary_emb.inv_freq', 'text_encoder.transformer.layers.1.self_attn.rotary_emb.inv_freq', 'text_encoder.transformer.layers.2.self_attn.rotary_emb.inv_freq', 'text_encoder.transformer.layers.3.self_attn.rotary_emb.inv

In [19]:
!ls ../configs/config_bert_large.json

../configs/config_bert_large.json


In [ ]:
texts, probs = retrieve_text(frames, text_candidates, model=intern_model, topk=5, config=config)

for t, p in zip(texts, probs):
    print(f'text: {t} ~ prob: {p:.4f}')